In [37]:
import pandas as pd
import pmdarima as pm

import os.path
from os import path
import numpy as np
import pickle
import lightgbm as lgb
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
from statsmodels.tsa.statespace.sarimax import SARIMAX
import datetime
%matplotlib inline

In [38]:
def smape_error(forecast, actual):
    numerator = np.absolute(forecast-actual)
    denominator = np.absolute(forecast) + np.absolute(actual)

    num_samples = len(numerator)

    return 100/num_samples * np.sum(numerator/denominator)

In [39]:
df = pd.read_pickle('1.collect_data/data_files/1hr_1block-average.pkl').replace([np.inf, -np.inf], np.nan).dropna()
df.reset_index(inplace=True)

In [40]:
blockface_detail = pd.read_csv('1.collect_data/blockface_detail.csv')

In [41]:
weather = pd.read_pickle('1.collect_data/data_files/weather/2010-2019_daily_weather.pkl')
weather = weather.resample('H').ffill()
weather.head()

,PRCP,SNOW,TAVG,TMAX,TMIN
DATE,,,,,
2010-01-01 00:00:00,0.36,0.0,48.5,52.0,45.0
2010-01-01 01:00:00,0.36,0.0,48.5,52.0,45.0
2010-01-01 02:00:00,0.36,0.0,48.5,52.0,45.0
2010-01-01 03:00:00,0.36,0.0,48.5,52.0,45.0
2010-01-01 04:00:00,0.36,0.0,48.5,52.0,45.0


In [42]:
df['isTrain'] = 0
train_column_index = df.columns.get_loc('isTrain') - 1

mask = df['OccupancyDateTime'] > ('2019-06-20') 
df.loc[mask, 'isTrain'] = 1


In [43]:
df.head()

,SourceElementKey,OccupancyDateTime,PaidOccupancy,ParkingSpaceCount,PercentOccupied,isTrain
0,1001,2012-01-03 08:00:00,4.316667,52.0,0.083013,0
1,1001,2012-01-03 09:00:00,9.933333,75.0,0.132444,0
2,1001,2012-01-03 10:00:00,11.850000,75.0,0.158000,0
3,1001,2012-01-03 11:00:00,18.033333,75.0,0.240444,0
4,1001,2012-01-03 12:00:00,24.183333,75.0,0.322444,0


In [44]:
train_column_index

4

In [45]:
block_dfs = []
for block in blockface_detail.sourceelementkey.values:
    block_file = 'batch_arima_results/%d_model.dat' % block
    if path.exists(block_file):

        # select only data for block, timeframe
        mask = (df['SourceElementKey'] == block) 
        curr = df[mask].set_index('OccupancyDateTime')
        block_dfs.append(pd.merge(curr, weather, left_index=True, right_index=True))
        


In [46]:
blocks_with_weather = pd.concat(block_dfs)
blocks_with_weather.index.name='OccupancyDateTime'
blocks_with_weather.head()

,SourceElementKey,PaidOccupancy,ParkingSpaceCount,PercentOccupied,isTrain,PRCP,SNOW,TAVG,TMAX,TMIN
OccupancyDateTime,,,,,,,,,,
2012-01-03 08:00:00,1001,4.316667,52.0,0.083013,0,0.02,0.0,48.0,53.0,43.0
2012-01-03 09:00:00,1001,9.933333,75.0,0.132444,0,0.02,0.0,48.0,53.0,43.0
2012-01-03 10:00:00,1001,11.850000,75.0,0.158000,0,0.02,0.0,48.0,53.0,43.0
2012-01-03 11:00:00,1001,18.033333,75.0,0.240444,0,0.02,0.0,48.0,53.0,43.0
2012-01-03 12:00:00,1001,24.183333,75.0,0.322444,0,0.02,0.0,48.0,53.0,43.0


In [47]:
merged = pd.merge(blocks_with_weather.reset_index(), 
                  blockface_detail, 
                  how='left', 
                  left_on='SourceElementKey', 
                  right_on='sourceelementkey').drop(['sourceelementkey', 
                                                     'location', 
                                                     'parkingcategory', 
                                                     'Unnamed: 0', 
                                                     'parkingtimelimitcategory'], axis=1)

In [48]:
merged.head()

,OccupancyDateTime,SourceElementKey,PaidOccupancy,ParkingSpaceCount,PercentOccupied,isTrain,PRCP,SNOW,TAVG,TMAX,TMIN,blockfacename,paidparkingarea,paidparkingsubarea,parkingspacecount,sideofstreet,latitude,longitude
0,2012-01-03 08:00:00,1001,4.316667,52.0,0.083013,0,0.02,0.0,48.0,53.0,43.0,1ST AVE BETWEEN CHERRY ST AND COLUMBIA ST,Pioneer Square,Core,4,SW,47.602873,-122.334694
1,2012-01-03 09:00:00,1001,9.933333,75.0,0.132444,0,0.02,0.0,48.0,53.0,43.0,1ST AVE BETWEEN CHERRY ST AND COLUMBIA ST,Pioneer Square,Core,4,SW,47.602873,-122.334694
2,2012-01-03 10:00:00,1001,11.850000,75.0,0.158000,0,0.02,0.0,48.0,53.0,43.0,1ST AVE BETWEEN CHERRY ST AND COLUMBIA ST,Pioneer Square,Core,4,SW,47.602873,-122.334694
3,2012-01-03 11:00:00,1001,18.033333,75.0,0.240444,0,0.02,0.0,48.0,53.0,43.0,1ST AVE BETWEEN CHERRY ST AND COLUMBIA ST,Pioneer Square,Core,4,SW,47.602873,-122.334694
4,2012-01-03 12:00:00,1001,24.183333,75.0,0.322444,0,0.02,0.0,48.0,53.0,43.0,1ST AVE BETWEEN CHERRY ST AND COLUMBIA ST,Pioneer Square,Core,4,SW,47.602873,-122.334694


In [49]:
merged['Month'] = merged.OccupancyDateTime.dt.month
merged['Year'] = merged.OccupancyDateTime.dt.year
merged['Hour'] = merged.OccupancyDateTime.dt.hour
merged['DayOfWeek'] = merged.OccupancyDateTime.dt.dayofweek
merged['Day'] = merged.OccupancyDateTime.dt.day
merged['IsWeekend'] = (merged.OccupancyDateTime.dt.dayofweek >= 5)

In [50]:
for col in ['SourceElementKey', 'DayOfWeek', 'Hour', 'paidparkingarea']:
    merged[col] = merged[col].astype('category')

In [51]:
mask = merged['isTrain'] == 1
df_train = merged[mask]
df_test = merged[~mask]

In [52]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1066526 entries, 26852 to 37675659
Data columns (total 24 columns):
OccupancyDateTime     1066526 non-null datetime64[ns]
SourceElementKey      1066526 non-null category
PaidOccupancy         1066526 non-null float64
ParkingSpaceCount     1066526 non-null float64
PercentOccupied       1066526 non-null float64
isTrain               1066526 non-null int64
PRCP                  1066526 non-null float64
SNOW                  1066526 non-null float64
TAVG                  1066526 non-null float64
TMAX                  1066526 non-null float64
TMIN                  1066526 non-null float64
blockfacename         1066526 non-null object
paidparkingarea       1066526 non-null category
paidparkingsubarea    795415 non-null object
parkingspacecount     1066526 non-null int64
sideofstreet          1066526 non-null object
latitude              1066526 non-null float64
longitude             1066526 non-null float64
Month                 1066526 non-n

In [53]:
rf_columns = ['SourceElementKey', 'Month', 'DayOfWeek', 
              'latitude', 'longitude', 'IsWeekend', 
              'paidparkingarea', 'Hour', 'Day',
              'PRCP',  'SNOW',  'TAVG',  
             ]

In [55]:
rf_train = lgb.Dataset(data=df_train[rf_columns], 
                           label=df_train['PercentOccupied'],
                            feature_name=rf_columns)
rf_test = lgb.Dataset(data=df_test[rf_columns], 
                          label=df_test['PercentOccupied'], 
                          feature_name=rf_columns,
                          reference=rf_train)

In [56]:
params = {
    'boosting_type': 'rf',
    'objective': 'regression',
    'metric': {'l2', 'l1'},
    'num_leaves': 1000,
    'learning_rate': 0.01,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'num_threads':6,
    'n_iter':1000,
    
}

In [57]:
print('Starting training...')
# train
gbm_rf = lgb.train(params,
                rf_train,
                valid_sets=rf_test)

print('Saving model...')
# save model to file
gbm_rf.save_model('only_rf_all_model.txt')

Starting training...


/Users/bethanys08/.pyenv/versions/anaconda3-2019.03/envs/myenv2/lib/python3.7/site-packages/lightgbm/engine.py:118: UserWarning: Found `n_iter` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/Users/bethanys08/.pyenv/versions/anaconda3-2019.03/envs/myenv2/lib/python3.7/site-packages/lightgbm/basic.py:762: UserWarning: categorical_feature in param dict is overridden.
  warnings.warn('categorical_feature in param dict is overridden.')


[1]	valid_0's l2: 0.0344887	valid_0's l1: 0.133127
[2]	valid_0's l2: 0.0329335	valid_0's l1: 0.131609
[3]	valid_0's l2: 0.0326152	valid_0's l1: 0.13023
[4]	valid_0's l2: 0.033542	valid_0's l1: 0.134888
[5]	valid_0's l2: 0.0350318	valid_0's l1: 0.139345
[6]	valid_0's l2: 0.0343673	valid_0's l1: 0.13721
[7]	valid_0's l2: 0.0353752	valid_0's l1: 0.140087
[8]	valid_0's l2: 0.0346739	valid_0's l1: 0.138397
[9]	valid_0's l2: 0.0354774	valid_0's l1: 0.140594
[10]	valid_0's l2: 0.0349935	valid_0's l1: 0.13919
[11]	valid_0's l2: 0.0346619	valid_0's l1: 0.138117
[12]	valid_0's l2: 0.0343951	valid_0's l1: 0.137257
[13]	valid_0's l2: 0.0341597	valid_0's l1: 0.136536
[14]	valid_0's l2: 0.0340352	valid_0's l1: 0.135988
[15]	valid_0's l2: 0.0339058	valid_0's l1: 0.135498
[16]	valid_0's l2: 0.0338126	valid_0's l1: 0.135107
[17]	valid_0's l2: 0.0341427	valid_0's l1: 0.136263
[18]	valid_0's l2: 0.0340034	valid_0's l1: 0.135843
[19]	valid_0's l2: 0.0338867	valid_0's l1: 0.135488
[20]	valid_0's l2: 0.0337

[159]	valid_0's l2: 0.0332741	valid_0's l1: 0.134114
[160]	valid_0's l2: 0.0332609	valid_0's l1: 0.134079
[161]	valid_0's l2: 0.0332529	valid_0's l1: 0.134048
[162]	valid_0's l2: 0.0332859	valid_0's l1: 0.134164
[163]	valid_0's l2: 0.0332775	valid_0's l1: 0.134133
[164]	valid_0's l2: 0.0332697	valid_0's l1: 0.134102
[165]	valid_0's l2: 0.0332567	valid_0's l1: 0.13406
[166]	valid_0's l2: 0.0332478	valid_0's l1: 0.134026
[167]	valid_0's l2: 0.0332277	valid_0's l1: 0.133981
[168]	valid_0's l2: 0.0332187	valid_0's l1: 0.133961
[169]	valid_0's l2: 0.0332052	valid_0's l1: 0.133926
[170]	valid_0's l2: 0.0331975	valid_0's l1: 0.133896
[171]	valid_0's l2: 0.0331854	valid_0's l1: 0.133858
[172]	valid_0's l2: 0.0331697	valid_0's l1: 0.133814
[173]	valid_0's l2: 0.0331645	valid_0's l1: 0.133801
[174]	valid_0's l2: 0.033195	valid_0's l1: 0.133911
[175]	valid_0's l2: 0.0331874	valid_0's l1: 0.133885
[176]	valid_0's l2: 0.0331793	valid_0's l1: 0.133857
[177]	valid_0's l2: 0.0331689	valid_0's l1: 0.13

[315]	valid_0's l2: 0.0329378	valid_0's l1: 0.13309
[316]	valid_0's l2: 0.0329543	valid_0's l1: 0.133149
[317]	valid_0's l2: 0.0329528	valid_0's l1: 0.133139
[318]	valid_0's l2: 0.0329512	valid_0's l1: 0.133129
[319]	valid_0's l2: 0.0329498	valid_0's l1: 0.13312
[320]	valid_0's l2: 0.0329664	valid_0's l1: 0.133181
[321]	valid_0's l2: 0.0329833	valid_0's l1: 0.133242
[322]	valid_0's l2: 0.0329992	valid_0's l1: 0.133301
[323]	valid_0's l2: 0.0329967	valid_0's l1: 0.133285
[324]	valid_0's l2: 0.0329927	valid_0's l1: 0.133274
[325]	valid_0's l2: 0.0330081	valid_0's l1: 0.133331
[326]	valid_0's l2: 0.0330073	valid_0's l1: 0.133322
[327]	valid_0's l2: 0.0330062	valid_0's l1: 0.133313
[328]	valid_0's l2: 0.0330051	valid_0's l1: 0.133303
[329]	valid_0's l2: 0.0330013	valid_0's l1: 0.133292
[330]	valid_0's l2: 0.0329973	valid_0's l1: 0.13328
[331]	valid_0's l2: 0.032992	valid_0's l1: 0.133266
[332]	valid_0's l2: 0.0329862	valid_0's l1: 0.133248
[333]	valid_0's l2: 0.0329738	valid_0's l1: 0.1332

[471]	valid_0's l2: 0.0330435	valid_0's l1: 0.133509
[472]	valid_0's l2: 0.0330421	valid_0's l1: 0.133501
[473]	valid_0's l2: 0.0330531	valid_0's l1: 0.133541
[474]	valid_0's l2: 0.0330492	valid_0's l1: 0.133528
[475]	valid_0's l2: 0.033046	valid_0's l1: 0.133519
[476]	valid_0's l2: 0.0330418	valid_0's l1: 0.133506
[477]	valid_0's l2: 0.0330397	valid_0's l1: 0.133498
[478]	valid_0's l2: 0.0330385	valid_0's l1: 0.133494
[479]	valid_0's l2: 0.0330369	valid_0's l1: 0.133486
[480]	valid_0's l2: 0.0330348	valid_0's l1: 0.133476
[481]	valid_0's l2: 0.033031	valid_0's l1: 0.133463
[482]	valid_0's l2: 0.033027	valid_0's l1: 0.133453
[483]	valid_0's l2: 0.0330244	valid_0's l1: 0.133447
[484]	valid_0's l2: 0.0330217	valid_0's l1: 0.133437
[485]	valid_0's l2: 0.033018	valid_0's l1: 0.133427
[486]	valid_0's l2: 0.0330293	valid_0's l1: 0.133467
[487]	valid_0's l2: 0.0330252	valid_0's l1: 0.133456
[488]	valid_0's l2: 0.0330235	valid_0's l1: 0.133447
[489]	valid_0's l2: 0.0330215	valid_0's l1: 0.1334

[627]	valid_0's l2: 0.033024	valid_0's l1: 0.133463
[628]	valid_0's l2: 0.0330225	valid_0's l1: 0.133456
[629]	valid_0's l2: 0.0330208	valid_0's l1: 0.133449
[630]	valid_0's l2: 0.0330187	valid_0's l1: 0.133442
[631]	valid_0's l2: 0.0330177	valid_0's l1: 0.133435
[632]	valid_0's l2: 0.0330263	valid_0's l1: 0.133466
[633]	valid_0's l2: 0.0330249	valid_0's l1: 0.133459
[634]	valid_0's l2: 0.0330222	valid_0's l1: 0.133451
[635]	valid_0's l2: 0.0330195	valid_0's l1: 0.133443
[636]	valid_0's l2: 0.0330166	valid_0's l1: 0.133434
[637]	valid_0's l2: 0.0330254	valid_0's l1: 0.133465
[638]	valid_0's l2: 0.033024	valid_0's l1: 0.133458
[639]	valid_0's l2: 0.0330223	valid_0's l1: 0.133451
[640]	valid_0's l2: 0.0330199	valid_0's l1: 0.133445
[641]	valid_0's l2: 0.0330167	valid_0's l1: 0.133435
[642]	valid_0's l2: 0.0330153	valid_0's l1: 0.133428
[643]	valid_0's l2: 0.033014	valid_0's l1: 0.133421
[644]	valid_0's l2: 0.0330223	valid_0's l1: 0.133451
[645]	valid_0's l2: 0.0330207	valid_0's l1: 0.133

[783]	valid_0's l2: 0.0329922	valid_0's l1: 0.133341
[784]	valid_0's l2: 0.0329904	valid_0's l1: 0.133336
[785]	valid_0's l2: 0.0329899	valid_0's l1: 0.133331
[786]	valid_0's l2: 0.0329894	valid_0's l1: 0.133327
[787]	valid_0's l2: 0.0329885	valid_0's l1: 0.133322
[788]	valid_0's l2: 0.0329953	valid_0's l1: 0.133346
[789]	valid_0's l2: 0.0329901	valid_0's l1: 0.133335
[790]	valid_0's l2: 0.0329894	valid_0's l1: 0.133331
[791]	valid_0's l2: 0.0329885	valid_0's l1: 0.133325
[792]	valid_0's l2: 0.0329872	valid_0's l1: 0.13332
[793]	valid_0's l2: 0.0329853	valid_0's l1: 0.133314
[794]	valid_0's l2: 0.0329844	valid_0's l1: 0.133309
[795]	valid_0's l2: 0.032982	valid_0's l1: 0.133303
[796]	valid_0's l2: 0.0329813	valid_0's l1: 0.133298
[797]	valid_0's l2: 0.0329797	valid_0's l1: 0.133294
[798]	valid_0's l2: 0.0329791	valid_0's l1: 0.13329
[799]	valid_0's l2: 0.0329773	valid_0's l1: 0.133284
[800]	valid_0's l2: 0.0329766	valid_0's l1: 0.13328
[801]	valid_0's l2: 0.032976	valid_0's l1: 0.13327

[939]	valid_0's l2: 0.0329566	valid_0's l1: 0.133192
[940]	valid_0's l2: 0.0329556	valid_0's l1: 0.133188
[941]	valid_0's l2: 0.0329614	valid_0's l1: 0.133209
[942]	valid_0's l2: 0.0329609	valid_0's l1: 0.133205
[943]	valid_0's l2: 0.0329606	valid_0's l1: 0.133201
[944]	valid_0's l2: 0.0329589	valid_0's l1: 0.133196
[945]	valid_0's l2: 0.0329582	valid_0's l1: 0.133191
[946]	valid_0's l2: 0.032957	valid_0's l1: 0.133187
[947]	valid_0's l2: 0.0329558	valid_0's l1: 0.133182
[948]	valid_0's l2: 0.0329549	valid_0's l1: 0.133179
[949]	valid_0's l2: 0.0329533	valid_0's l1: 0.133174
[950]	valid_0's l2: 0.0329522	valid_0's l1: 0.13317
[951]	valid_0's l2: 0.0329514	valid_0's l1: 0.133166
[952]	valid_0's l2: 0.0329496	valid_0's l1: 0.133161
[953]	valid_0's l2: 0.0329486	valid_0's l1: 0.133157
[954]	valid_0's l2: 0.0329444	valid_0's l1: 0.133148
[955]	valid_0's l2: 0.0329437	valid_0's l1: 0.133144
[956]	valid_0's l2: 0.032942	valid_0's l1: 0.133139
[957]	valid_0's l2: 0.0329412	valid_0's l1: 0.133

In [58]:
y_pred = gbm_rf.predict(merged[rf_columns], num_iteration=gbm_rf.best_iteration)

merged['RF_results'] = y_pred

In [61]:
arima_rf_train_smapes = []
arima_rf_test_smapes = []
for i, block in enumerate(blockface_detail.sourceelementkey.values):
    block_file = 'batch_arima_results/%d_model.dat' % block
    if path.exists(block_file):
        mask_train = (merged['SourceElementKey'] == block) & (merged['isTrain'] == 1)
        mask_test = (merged['SourceElementKey'] == block) & (merged['isTrain'] == 0)
        curr_train = merged[mask_train]
        curr_test = merged[mask_test]
        if len(curr_train) > 0 and len(curr_test) > 0:

            arima_rf_train_smapes.append( smape_error(curr_train.RF_results, curr_train.PercentOccupied))
            arima_rf_test_smapes.append( smape_error(curr_test.RF_results, curr_test.PercentOccupied))

In [62]:
print(np.mean(arima_rf_train_smapes), np.mean(arima_rf_test_smapes))

11.36727247025708 17.577802417825833
